## Rodando os modelos para uma coleção de imagens Landsat 

#### Este exemplo apresenta como iniciar o modelo geesebal e criar uma classe "Collection" para uma coleção de imagens Landsat utilizando a função ".overpass()"

### Importando a biblioteca Google Earth Engine

In [1]:
try:
    import ee 
except:
    !pip install earthengine-api
    import ee

In [2]:
# Instalando bibliotecas secundárias do OpenET

# Installing openet-core package
# https://github.com/Open-ET/openet-core

#    !pip install openet-core

# Installing openet.refetgee package
# https://github.com/Open-ET/openet-refet-gee

#    !pip install openet-refet-gee

### Inicializando Google Eaeth Engine
#### Necessário definir o projeto_id a ser utilizado

In [3]:
project_id = 'et-brasil'

try:
    ee.Initialize(project=project_id)
except:
    ee.Authenticate()

### Importando o modelo geeSEBAL e definido a localização do ambiente de trabalho 

In [4]:
import pprint
import sys
import datetime
import pandas as pd
from IPython.display import Image

sys.path.append(r'C:\openet_brazil\workshop_et_models\openet-geesebal')

import openet.geesebal as model

### Variáveis

In [10]:
collections = ['LANDSAT/LC08/C02/T1_L2','LANDSAT/LE07/C02/T1_L2']

meteorology_source_inst = "ECMWF/ERA5_LAND/HOURLY"
meteorology_source_daily = "projects/openet/assets/meteorology/era5land/sa/daily"
et_reference_source = 'projects/et-brasil/assets/era5/daily/etr'
et_reference_band = 'eto24h'
et_reference_factor = 1.0
et_reference_resample = 'nearest'
et_reference_date_type = 'daily'

# Ponto de interesse
point = ee.Geometry.Point([-47.2425,-17.5372])

# Período de análise
start_date = '2018-05-01'
end_date = '2018-07-01'

# Cobertura de nuvens
cloud_cover = 20

# Paleta de cores
ndvi_palette = ['#EFE7E1', '#003300']
et_palette = [
    'DEC29B', 'E6CDA1', 'EDD9A6', 'F5E4A9', 'FFF4AD', 'C3E683', '6BCC5C', 
    '3BB369', '20998F', '1C8691', '16678A', '114982', '0B2C7A']
viridis_palette = ['440154', '433982', '30678D', '218F8B', '36B677', '8ED542', 'FDE725']
lst_palette = ['blue','green','yellow','orange','red']

# Geometria para visualização
area_de_interesse = point.buffer(20000)

# Tamanho da imagem para a visualização
image_size = 768

### Criando o objeto do modelo

In [11]:
model_obj = model.Collection(
collections=collections,
start_date=start_date,
end_date=end_date,
geometry=point,
cloud_cover_max=cloud_cover,
model_args={
            'meteorology_source_inst': meteorology_source_inst,
            'meteorology_source_daily': meteorology_source_daily,
            'et_reference_source': et_reference_source,
            'et_reference_band': et_reference_band,
            'et_reference_factor': et_reference_factor,
            'et_reference_resample': et_reference_resample},) 

pprint.pprint(model_obj.get_image_ids())

['LANDSAT/LC08/C02/T1_L2/LC08_220072_20180501',
 'LANDSAT/LC08/C02/T1_L2/LC08_220072_20180602',
 'LANDSAT/LC08/C02/T1_L2/LC08_220072_20180618',
 'LANDSAT/LE07/C02/T1_L2/LE07_220072_20180509',
 'LANDSAT/LE07/C02/T1_L2/LE07_220072_20180610',
 'LANDSAT/LE07/C02/T1_L2/LE07_220072_20180626']


### Opção 1 para exportar uma tabela com os resultados para um ponto (Utilizando Pandas)
#### Obs: funciona para períodos curtos e modelos simples (ssebop, ptjpl, sims)

In [12]:
def get_region_df(info):
    """Convert the output of getRegions to a pandas dataframe"""
    col_dict = {}
    info_dict = {}
    for i, k in enumerate(info[0][4:]):
        col_dict[k] = i+4
        info_dict[k] = {}
        
    for row in info[1:]:
        date = datetime.datetime.utcfromtimestamp(row[3] / 1000.0).strftime('%Y-%m-%d')
        for k, v in col_dict.items():
            info_dict[k][date] = row[col_dict[k]]
            
    return pd.DataFrame.from_dict(info_dict).sort_index()

### Criando uma classe com os resultados de et, ndvi, etr e etf

In [13]:
overpass_coll = model_obj.overpass(variables=['lst', 'ndvi', 'et', 'et_fraction'])

### Visualizando os resultados em uma tabela

#### Para salvar utilizar a função .to_excel() ou .to_csv()

In [14]:
overpass_df = get_region_df(overpass_coll.getRegion(point, scale=30).getInfo())
pprint.pprint(overpass_df)


                   lst      ndvi        et  et_fraction
2018-05-01  299.780701  0.837812  2.553833     0.739750
2018-05-09  298.077301  0.818078  1.859841     0.581607
2018-06-02  281.856201  0.760818  3.476103     1.159744
2018-06-10  299.775269  0.724088  1.456040     0.496263
2018-06-18  297.636230  0.701364  1.782803     0.568610
2018-06-26  300.099670  0.585676  1.121821     0.434263


### Opção 2 para exportar uma tabela com os resultados para um ponto (Utilizando Earth Engine)
#### Para períodos mais longos e modelos mais complexos (geesebal, eemetric)


In [15]:
### Função GEE para extrair dados em uma tabela

def get_table(img):
    
    reduce= img.reduceRegion(
        reducer=ee.Reducer.mean(),
        scale=90,
        maxPixels=1e12,
        geometry=point)

    date= ee.Date(img.get('system:time_start')).format('YYYY-MM-dd')
    
    return ee.Feature(None, {
    'date' : date,
    'et': ee.List([reduce.get('et'),-9999]).reduce(ee.Reducer.firstNonNull()),
    'ndvi': ee.List([reduce.get('ndvi'),-9999]).reduce(ee.Reducer.firstNonNull()),
    'et_fraction ': ee.List([reduce.get('et_fraction'),-9999]).reduce(ee.Reducer.firstNonNull()),
    })


In [16]:
task = ee.batch.Export.table.toDrive(
    collection=overpass_coll.map(get_table),
    description='serie_de_dados_et',
    folder='workshop_ana',
    fileFormat='CSV'
)
task.start()